# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 8:33AM,256661,16,SHL-02062023,"SHL Pharma, LLC",Released
1,Feb 6 2023 8:30AM,256660,20,9123067,"Andelyn Biosciences, Inc.",Released
2,Feb 6 2023 8:25AM,256659,19,TR-RG-02062023,"GCH Granules USA, Inc.",Released
3,Feb 6 2023 8:22AM,256658,19,9123052,"Emersa Waterbox, LLC",Released
4,Feb 6 2023 8:11AM,256657,20,9123063,Else Nutrition,Released
5,Feb 6 2023 7:57AM,256656,19,ADV80011420,"AdvaGen Pharma, Ltd",Released
6,Feb 6 2023 7:57AM,256656,19,ADV80011421,"AdvaGen Pharma, Ltd",Released
7,Feb 6 2023 7:57AM,256656,19,ADV80011422,"AdvaGen Pharma, Ltd",Released
8,Feb 6 2023 7:56AM,256655,19,OPUS-104609,Opus Life Sciences,Released
9,Feb 6 2023 7:56AM,256655,19,OPUS-104654,Opus Life Sciences,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
5,256657,Released,1
6,256658,Released,1
7,256659,Released,1
8,256660,Released,1
9,256661,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256657,NaN,1.0
256658,NaN,1.0
256659,NaN,1.0
256660,NaN,1.0
256661,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256651,1.0,0.0
256652,0.0,11.0
256654,0.0,1.0
256655,0.0,2.0
256656,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256651,1,0
256652,0,11
256654,0,1
256655,0,2
256656,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256651,1,0
1,256652,0,11
2,256654,0,1
3,256655,0,2
4,256656,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256651,1,
1,256652,,11
2,256654,,1
3,256655,,2
4,256656,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC"
1,Feb 6 2023 8:30AM,256660,20,"Andelyn Biosciences, Inc."
2,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc."
3,Feb 6 2023 8:22AM,256658,19,"Emersa Waterbox, LLC"
4,Feb 6 2023 8:11AM,256657,20,Else Nutrition
5,Feb 6 2023 7:57AM,256656,19,"AdvaGen Pharma, Ltd"
8,Feb 6 2023 7:56AM,256655,19,Opus Life Sciences
10,Feb 6 2023 7:52AM,256654,19,"Emersa Waterbox, LLC"
11,Feb 6 2023 7:43AM,256652,19,"Innogenix, LLC"
22,Feb 6 2023 7:16AM,256651,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC",,1
1,Feb 6 2023 8:30AM,256660,20,"Andelyn Biosciences, Inc.",,1
2,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc.",,1
3,Feb 6 2023 8:22AM,256658,19,"Emersa Waterbox, LLC",,1
4,Feb 6 2023 8:11AM,256657,20,Else Nutrition,,1
5,Feb 6 2023 7:57AM,256656,19,"AdvaGen Pharma, Ltd",,3
6,Feb 6 2023 7:56AM,256655,19,Opus Life Sciences,,2
7,Feb 6 2023 7:52AM,256654,19,"Emersa Waterbox, LLC",,1
8,Feb 6 2023 7:43AM,256652,19,"Innogenix, LLC",,11
9,Feb 6 2023 7:16AM,256651,22,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC",1,
1,Feb 6 2023 8:30AM,256660,20,"Andelyn Biosciences, Inc.",1,
2,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc.",1,
3,Feb 6 2023 8:22AM,256658,19,"Emersa Waterbox, LLC",1,
4,Feb 6 2023 8:11AM,256657,20,Else Nutrition,1,
5,Feb 6 2023 7:57AM,256656,19,"AdvaGen Pharma, Ltd",3,
6,Feb 6 2023 7:56AM,256655,19,Opus Life Sciences,2,
7,Feb 6 2023 7:52AM,256654,19,"Emersa Waterbox, LLC",1,
8,Feb 6 2023 7:43AM,256652,19,"Innogenix, LLC",11,
9,Feb 6 2023 7:16AM,256651,22,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC",1,
1,Feb 6 2023 8:30AM,256660,20,"Andelyn Biosciences, Inc.",1,
2,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc.",1,
3,Feb 6 2023 8:22AM,256658,19,"Emersa Waterbox, LLC",1,
4,Feb 6 2023 8:11AM,256657,20,Else Nutrition,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC",1.0,NaN
1,Feb 6 2023 8:30AM,256660,20,"Andelyn Biosciences, Inc.",1.0,NaN
2,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc.",1.0,NaN
3,Feb 6 2023 8:22AM,256658,19,"Emersa Waterbox, LLC",1.0,NaN
4,Feb 6 2023 8:11AM,256657,20,Else Nutrition,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 8:33AM,256661,16,"SHL Pharma, LLC",1.0,0.0
1,Feb 6 2023 8:30AM,256660,20,"Andelyn Biosciences, Inc.",1.0,0.0
2,Feb 6 2023 8:25AM,256659,19,"GCH Granules USA, Inc.",1.0,0.0
3,Feb 6 2023 8:22AM,256658,19,"Emersa Waterbox, LLC",1.0,0.0
4,Feb 6 2023 8:11AM,256657,20,Else Nutrition,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
16,256661,1.0,0.0
19,1539934,19.0,0.0
20,513317,2.0,0.0
22,256651,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,16,256661,1.0,0.0
1,19,1539934,19.0,0.0
2,20,513317,2.0,0.0
3,22,256651,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,16,1.0,0.0
1,19,19.0,0.0
2,20,2.0,0.0
3,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,16,Released,1.0
1,19,Released,19.0
2,20,Released,2.0
3,22,Released,0.0
4,16,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,16,19,20,22
Status,,,,
Executing,0.0,0.0,0.0,1.0
Released,1.0,19.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,16,19,20,22
0,Executing,0.0,0.0,0.0,1.0
1,Released,1.0,19.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,16,19,20,22
0,Executing,0.0,0.0,0.0,1.0
1,Released,1.0,19.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()